# Pipeline de preparación de datos para fine-tuning con LoRA

Este notebook documentado contiene el pipeline completo para cargar, limpiar y preparar datos con el objetivo de realizar un fine-tuning de modelos de lenguaje usando la técnica adaptaciones LoRA. Cada bloque de código va precedido por una celda Markdown que explica el propósito, qué hace y por qué es importante (justificación), además de consideraciones prácticas.

Objetivos generales:

- Evitar *data leakage* mediante divisiones correctas y control de información temporal.
- Asegurar la calidad de los datos revisando su estructura interna y sondeando todos los datos.
- Formatear y tokenizar los datos en el formato requerido por el modelo y por LoRA/PEFT.

### Paso 1: Configuración e importación de librerías

**Objetivo:** Este bloque importa las librerías y configura el entorno necesario para el pipeline.

In [1]:
import json
import os
import re
from typing import List, Dict, Tuple, Set

In [ ]:
FILE_TRAIN = "fine_tuning_train_500_dataset.json"
FILE_VAL = "fine_tuning_50_test_dataset.json"


CLASS_NO_DUPLICATE = "✔️ No duplicates detected"
CLASS_DUPLICATE_PREFIX = "❗ Issue may be repeated or similar to UCM"

TEMPLATE_PARTS = [
    "Eres un meticuloso analista de datos experto en Jira.",
    "**ISSUE EN ANÁLISIS AHORA:**",
    "---",
    "### PROCESO DE RAZONAMIENTO Y REGLAS",
    "**Paso 1: Tarea Única - Detección de Duplicados.**",
    "**Paso 2: Generar el Payload Combinado.**",
    "---",
    "### DATOS PARA TU ANÁLISIS",
    "**1. CURRENT ISSUE (El que estás analizando ahora):**",
    "**2. POTENTIALLY SIMILAR ISSUES (Encontrados en la memoria vectorial):**",
    "---",
    "### REGLAS DE PAYLOAD",
    "* **FORMATO:**",
    "* **PROHIBICIÓN ABSOLUTA:**",
    "* **CAMPOS PROHIBIDOS:**"
]

**Paso 2: Eliminación de duplicados**

- **Objetivo:** Detectar y eliminar duplicados exactos y redundancias contextuales en el dataset para evitar `data leakage` y sobreajuste antes de dividir los datos en `train`/`val`/`test`.

- **Resumen (qué hace el código):**
  - `get_unique_key(item)`: crea una clave hashable `(input, output)` para detectar duplicados exactos.
  - `validate_input_template(input_text)`: verifica que el prompt siga la estructura estática definida en `TEMPLATE_PARTS`.
  - `detect_context_redundancy(input_text)`: analiza la sección `POTENTIALLY SIMILAR ISSUES` buscando pares `Summary/Description` repetidos dentro del mismo prompt.
  - `process_dataset(file_path)`: orquesta verificaciones (integridad JSON, estructura, plantilla, redundancia y duplicados internos) y devuelve `valid_data` listo para análisis posterior.
  - `analyze_data_distribution(data, file_name)`: calcula la distribución de clases (`No Duplicates` vs `Duplicates`) e imprime alertas si hay desbalance.
  - `check_cross_contamination(train_data, val_data)`: compara `train` y `val` para detectar ejemplos idénticos (fuga de datos).

- **Salida y mensajes esperados:**
  - Mensajes informativos como `PROCESANDO: <file>` y resúmenes `Resultados del Análisis:`.
  - Advertencias `⚠️` para errores de plantilla, ítems con contexto redundante, o duplicados internos.
  - `CRÍTICO` si se detecta contaminación cruzada entre `train` y `val`.

- **Justificación:** Los duplicados y la redundancia en el contexto inflan el rendimiento aparente y permiten que el modelo memorice ejemplos en lugar de generalizar; eliminarlos o marcarlos reduce el overfitting y evita métricas optimistas en validación.

- **Consideraciones y recomendaciones:**
  - Mantener reproducibilidad: fijar semillas y versionado de librerías antes de procesar.
  - No eliminar automáticamente near-duplicates sin revisión humana; considerar hashing difuso o reglas de umbral si procede.
  - Revisar y corregir ejemplos con errores de `TEMPLATE_PARTS` antes de descartarlos.
  - Si el dataset queda desbalanceado, aplicar `upsampling` o `downsampling` según corresponda.

---

In [ ]:
def get_unique_key(item: Dict) -> Tuple[str, str]:
    """Genera una tupla hashable (input, output) para detectar duplicados exactos."""
    try:
        input_str = item['input']
        output_str = json.dumps(item['output'], sort_keys=True, ensure_ascii=False)
        return (input_str, output_str)
    except (KeyError, TypeError):
        return None

def validate_input_template(input_text: str) -> bool:
    """Verifica que el prompt cumpla con la estructura estática requerida."""
    current_position = 0
    for part in TEMPLATE_PARTS:
        found_position = input_text.find(part, current_position)
        if found_position == -1:
            return False
        current_position = found_position + len(part)
    return True

def detect_context_redundancy(input_text: str) -> int:
    """
    NUEVA FUNCIÓN: Analiza la sección 'POTENTIALLY SIMILAR ISSUES' 
    para detectar si hay Summary/Description repetidos dentro del mismo prompt.
    
    Retorna: Número de duplicados encontrados en este prompt específico.
    """
   
    start_marker = "**2. POTENTIALLY SIMILAR ISSUES (Encontrados en la memoria vectorial):**"
    end_marker = "---" 
    
    start_idx = input_text.find(start_marker)
    if start_idx == -1: return 0
    
    
    sub_text = input_text[start_idx + len(start_marker):]
    
    
    end_idx = sub_text.find(end_marker)
    if end_idx != -1:
        sub_text = sub_text[:end_idx]
        
    pattern = re.compile(r"Summary:\s*(.+?)\n\s*Description:\s*(.+?)(?=\n- ISSUE|\n\n|\n---|$)", re.IGNORECASE)
    
    matches = pattern.findall(sub_text)
    
    seen_content = set()
    redundancy_count = 0
    
    for summary, description in matches:
        
        clean_pair = (summary.strip(), description.strip())
        
        if clean_pair in seen_content:
            redundancy_count += 1
        else:
            seen_content.add(clean_pair)
            
    return redundancy_count

def analyze_data_distribution(data: List[Dict], file_name: str):
    """Analiza balance de clases (Duplicado vs No Duplicado)."""
    print(f"\n  [Distribución] Análisis del archivo: {os.path.basename(file_name)}")
    counts = {"no_duplicate": 0, "duplicate": 0, "unknown": 0}
    
    for item in data:
        try:
            value = item['output']['customfield_10602']
            if value.strip() == CLASS_NO_DUPLICATE:
                counts["no_duplicate"] += 1
            elif value.strip().startswith(CLASS_DUPLICATE_PREFIX):
                counts["duplicate"] += 1
            else:
                counts["unknown"] += 1
        except (KeyError, TypeError):
            counts["unknown"] += 1 

    total = len(data)
    if total == 0: return

    p_no_dup = (counts["no_duplicate"] / total) * 100
    p_dup = (counts["duplicate"] / total) * 100
    
    print(f"   - Total ejemplos: {total}")
    print(f"   - 'No Duplicates': {counts['no_duplicate']} ({p_no_dup:.1f}%)")
    print(f"   - 'Duplicates':    {counts['duplicate']} ({p_dup:.1f}%)")
    
    if counts["unknown"] > 0:
        print(f"   - Formato desconocido: {counts['unknown']}")

    if not (0.4 <= (p_no_dup / 100) <= 0.6):
         print("   - ALERTA: Dataset desbalanceado. Recomendado técnica de upsampling/downsampling.")
    else:
         print("   - ✅ Dataset balanceado.")

**Paso 3: Carga y validación inicial de datos**

- **Objetivo:** Cargar el archivo fuente (`JSON`, `CSV` o `Excel`) y ejecutar las comprobaciones iniciales necesarias para garantizar que los datos están en el formato esperado antes de cualquier preparación adicional o división en `train/val/test`.

- **Qué hace el código que sigue (resumen):**
  - `process_dataset(file_path)`: función orquestadora que realiza:

- **Entradas esperadas:**
  - `file_path` (string): ruta a un archivo JSON que contiene una lista de ejemplos con la forma `{"input": <str>, "output": <dict>}`.
    4) Detección de redundancia contextual dentro de la sección `POTENTIALLY SIMILAR ISSUES` (`detect_context_redundancy`).
- **Salidas y efectos:**
  - Retorna `valid_data`: lista de ítems que pasaron las comprobaciones básicas (lista de `dict`).
  - Imprime mensajes informativos y alertas: `PROCESANDO`, `Resultados del Análisis`.
  - Actualiza un log interno en memoria (`issues_log`) con conteos de errores de estructura, errores de plantilla, duplicados internos y redundancias contextuales.

---  - Si se pretende usar formatos distintos a JSON, añadir adaptadores de lectura (p. ej. `pandas.read_csv`) antes de llamar a `process_dataset` o extender la función para soportarlos.  - Mantener backups de los archivos originales para poder revertir filtrados o correcciones manuales.  - Ejecutar `process_dataset` en local y revisar las primeras advertencias (`template_errors`) antes de borrar datos automáticamente.- **Recomendaciones prácticas:**- **Justificación:** Centralizar la carga y las comprobaciones evita que datos malformados contaminen etapas posteriores (tokenización, entrenamiento), y permite detectar fugas de información o memorias innecesarias antes de entrenar.  - Si se detecta `context_redundancy`, puede indicar que la sección de `POTENTIALLY SIMILAR ISSUES` contiene ejemplos repetidos y conviene depurarlos para evitar sesgos en RAG/recuperación.  - Plantillas de prompt que no encajan con `TEMPLATE_PARTS` se cuentan como `template_errors` (revisar y corregir manualmente si procede).  - Ítems con estructura distinta a `{'input','output'}` son contabilizados y saltados.  - Archivo inexistente o JSON corrupto (se captura y se informa).- **Errores y condiciones a vigilar:**
**Justificación / razones:** Justificación: centralizar y documentar la carga de datos ayuda a comprobar formatos, detectar encabezados incorrectos y prevenir fugas de información (data leakage) durante posteriores pasos de preparación.

**Consideraciones:** Consideraciones: revisar que las transformaciones no introduzcan información del futuro, anotar supuestos, y mantener reproducibilidad (semillas, versiones de librerías).

---

In [ ]:
def process_dataset(file_path: str) -> List[Dict]:
    """
    Ejecuta todas las verificaciones sobre un archivo:
    1. Integridad JSON
    2. Estructura de campos
    3. Duplicados internos (Data Leakage intra-set)
    4. Validación de Plantilla (Prompt Template)
    5. Redundancia de Contexto (RAG repetition)
    """
    print(f"\n{'='*60}")
    print(f"PROCESANDO: {file_path}")
    print(f"{'='*60}")

    if not os.path.exists(file_path):
        print(f"Error: Archivo no encontrado.")
        return None
        
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error: JSON corrupto o ilegible. {e}")
        return None

    issues_log = {
        "structure_errors": 0,
        "template_errors": 0,
        "internal_duplicates": 0,
        "context_redundancy_items": 0 
    }
    
    valid_data = []
    

    for i, item in enumerate(data):
    
        if set(item.keys()) != {'input', 'output'}:
            issues_log["structure_errors"] += 1
            continue
            
        input_str = item.get('input', "")
        

        if not validate_input_template(input_str):
            issues_log["template_errors"] += 1
            if issues_log["template_errors"] == 1:
                print(f"Ejemplo de error de plantilla en índice {i}.")
        

        redundancy_count = detect_context_redundancy(input_str)
        if redundancy_count > 0:
            issues_log["context_redundancy_items"] += 1
            print(f"Ítem {i} tiene {redundancy_count} issues repetidos en su contexto.") 

        valid_data.append(item)

    unique_set = set()
    for item in valid_data:
        key = get_unique_key(item)
        if key:
            if key in unique_set:
                issues_log["internal_duplicates"] += 1
            else:
                unique_set.add(key)
    
    print(f"Resultados del Análisis:")
    if sum(issues_log.values()) == 0:
        print("Estructura, Template, Ningún error de duplicados internos. (Limpieza correcta)")
    else:
        if issues_log["structure_errors"]: print(f"Errores de estructura JSON: {issues_log['structure_errors']}")
        if issues_log["template_errors"]: print(f"Errores en plantilla de prompt: {issues_log['template_errors']}")
        if issues_log["internal_duplicates"]: print(f"Duplicados exactos internos: {issues_log['internal_duplicates']} (Deben eliminarse)")
        if issues_log["context_redundancy_items"]: 
            print(f"Items con contexto redundante: {issues_log['context_redundancy_items']}")
            print(f"(Esto significa que en 'POTENTIALLY SIMILAR ISSUES' aparecen issues repetidos)")

    analyze_data_distribution(valid_data, file_path)
    
    return valid_data

def check_cross_contamination(train_data, val_data):
    """Verifica si hay datos de validación filtrados en entrenamiento."""
    print("\n Verificando Fuga de Datos (Train vs Validation)...")
    if not train_data or not val_data:
        print("No se puede verificar, faltan datos.")
        return

    train_hashes = {get_unique_key(item) for item in train_data if get_unique_key(item)}
    leakage_count = 0
    
    for item in val_data:
        if get_unique_key(item) in train_hashes:
            leakage_count += 1
            
    if leakage_count > 0:
        print(f"CRÍTICO: Se encontraron {leakage_count} ejemplos de validación dentro de train.")
    else:
        print("Limpio. No hay contaminación cruzada.")

**Ejecución: Carga, validación y verificación de fuga de datos**

Este bloque ejecuta las comprobaciones completas del pipeline sobre los archivos fuente y, si ambos se procesan correctamente, verifica si hay ejemplos idénticos entre `train` y `val` (fuga de datos).

- **Qué hace exactamente:**
  1. `train_dataset = process_dataset(FILE_TRAIN)` -> Lee `FILE_TRAIN`, valida estructura, plantilla y duplicados internos, y retorna `valid_data` (lista de `dict`) o `None` si hay un error crítico.
  2. `val_dataset = process_dataset(FILE_VAL)` -> Igual verificación para el conjunto de validación.
  3. `if train_dataset and val_dataset: check_cross_contamination(train_dataset, val_dataset)` -> Si ambos conjuntos se cargaron correctamente, calcula hashes únicos por ejemplo y cuenta cuántos de los ejemplos de `val` aparecen en `train`. Imprime un mensaje crítico si se detecta contaminación.

- **Salidas esperadas en la consola:**
  - Mensajes `PROCESANDO: <file>` mientras se leen los ficheros.
  - Resumen `Resultados del Análisis:` con conteos de `structure_errors`, `template_errors`, `internal_duplicates` y `context_redundancy_items`.
  - Si hay contaminación cruzada: `CRÍTICO: Se encontraron <n> ejemplos de validación dentro de train.`
  - Si no hay contaminación: `Limpio. No hay contaminación cruzada.`

- **Acciones recomendadas según resultado:**
  - Si `process_dataset` devuelve `None` para cualquiera de los archivos: revisar el error impreso (archivo no encontrado o JSON corrupto) y corregir el origen antes de continuar.
  - Si hay `template_errors`: abrir algunos ejemplos reportados y corregir manualmente la sección del prompt o ajustar `TEMPLATE_PARTS` si corresponde.
  - Si hay `internal_duplicates`: eliminar o consolidar ejemplos duplicados antes de entrenar para evitar sesgos.
  - Si se detecta contaminación cruzada (`leakage_count > 0`): auditar los ejemplos coincidentes (por ejemplo, buscar `get_unique_key(item)` para los ítems dañinos) y eliminar/reestruturar los datos para garantizar independencia entre `train` y `val`.


---

In [ ]:
train_dataset = process_dataset(FILE_TRAIN)
val_dataset = process_dataset(FILE_VAL)
if train_dataset and val_dataset:
    check_cross_contamination(train_dataset, val_dataset)


PROCESANDO: fine_tuning_train_500_dataset.json
Resultados del Análisis:
Estructura, Template, Ningún error de duplicados internos. (Limpieza correcta)

  [Distribución] Análisis del archivo: fine_tuning_train_500_dataset.json
   - Total ejemplos: 500
   - 'No Duplicates': 231 (46.2%)
   - 'Duplicates':    269 (53.8%)
   - ✅ Dataset balanceado.

PROCESANDO: fine_tuning_50_test_dataset.json
Resultados del Análisis:
Estructura, Template, Ningún error de duplicados internos. (Limpieza correcta)

  [Distribución] Análisis del archivo: fine_tuning_50_test_dataset.json
   - Total ejemplos: 53
   - 'No Duplicates': 27 (50.9%)
   - 'Duplicates':    26 (49.1%)
   - ✅ Dataset balanceado.

 Verificando Fuga de Datos (Train vs Validation)...
Limpio. No hay contaminación cruzada.
